In [3]:
import os
from Bio.PDB import PDBParser, Superimposer
from Bio.PDB.NeighborSearch import NeighborSearch

def parse_structure(pdb_file, structure_id="structure"):
    """
    Parse a PDB file and return the first model of the structure.
    """
    parser = PDBParser(QUIET=True)
    structure = parser.get_structure(structure_id, pdb_file)
    return structure[0]  # return the first model (common for X-ray structures)

def get_ca_atoms(model, chain_id="A"):
    """
    Retrieve the alpha-carbon (Cα) atoms from a specific chain.
    Return a list of Bio.PDB.Atom objects.
    """
    ca_atoms = []
    if chain_id not in model:
        raise ValueError(f"Chain {chain_id} not found in structure.")
    chain = model[chain_id]
    for residue in chain:
        if residue.has_id("CA"):
            ca_atoms.append(residue["CA"])
    return ca_atoms

def get_ligand_atoms(model, chain_id="A", ligand_name="ATP"):
    """
    Retrieve all atoms corresponding to a specified ligand (e.g., 'ATP') in the given chain.
    Returns a list of Atom objects. Returns an empty list if not found.
    """
    ligand_atoms = []
    if chain_id not in model:
        return ligand_atoms

    chain = model[chain_id]
    for residue in chain:
        if residue.get_resname().strip() == ligand_name:
            for atom in residue:
                ligand_atoms.append(atom)
    return ligand_atoms

def rmsd_atoms(atoms_ref, atoms_mov):
    """
    Compute the RMSD between two lists of atoms (after they're superimposed).
    """
    import numpy as np

    if len(atoms_ref) != len(atoms_mov):
        raise ValueError("Atom lists must have the same length for RMSD calculation.")

    sq_dist_sum = 0.0
    for a_ref, a_mov in zip(atoms_ref, atoms_mov):
        diff = a_ref.coord - a_mov.coord
        sq_dist_sum += (diff * diff).sum()

    return (sq_dist_sum / len(atoms_ref)) ** 0.5

def main():
    # --------------------------------------
    # 1. File paths (change these as needed)
    # --------------------------------------
    # Suppose you downloaded the structure for 5P21 (Ras) as "5P21.pdb"
    experimental_pdb = "5P21.pdb"  # Contains chain A with GTP, in reality
    predicted_pdb = "predicted_model.pdb"  # Your predicted model (with or without ATP)

    chain_id = "A"        # Adjust if your chain is labeled differently
    ligand_name = "ATP"
    # --------------------------------------
    # 2. Parse the Structures
    # --------------------------------------
    exp_model = parse_structure(experimental_pdb, structure_id="exp")
    pred_model = parse_structure(predicted_pdb, structure_id="pred")

    # --------------------------------------
    # 3. Get Protein Cα Atoms for Alignment
    # --------------------------------------
    exp_ca = get_ca_atoms(exp_model, chain_id=chain_id)
    pred_ca = get_ca_atoms(pred_model, chain_id=chain_id)

    if not exp_ca or not pred_ca:
        raise ValueError("No Cα atoms found in one or both structures.")

    # --------------------------------------
    # 4. Superimpose Predicted onto Experimental
    # --------------------------------------
    sup = Superimposer()
    sup.set_atoms(exp_ca, pred_ca)
    # Apply the rotation/translation to the entire predicted model
    sup.apply(pred_model.get_atoms())
    print(f"Protein backbone RMSD (Cα) after alignment: {sup.rms:.3f} Å")

    # --------------------------------------
    # 5. Compare Ligand Positions (ATP/GTP)
    # --------------------------------------
    exp_ligand_atoms = get_ligand_atoms(exp_model, chain_id=chain_id, ligand_name=ligand_name)
    pred_ligand_atoms = get_ligand_atoms(pred_model, chain_id=chain_id, ligand_name=ligand_name)

    if not exp_ligand_atoms:
        print(f"No {ligand_name} found in experimental structure. Check if it's really there.")
        return

    if not pred_ligand_atoms:
        print(f"No {ligand_name} found in predicted structure. Cannot compare ligand positions.")
        return

    if len(exp_ligand_atoms) != len(pred_ligand_atoms):
        print("Mismatch in ligand atom counts between experimental and predicted.")
        return

    # Calculate RMSD for ligand
    ligand_rmsd = rmsd_atoms(exp_ligand_atoms, pred_ligand_atoms)
    print(f"{ligand_name} RMSD (experimental vs. predicted): {ligand_rmsd:.3f} Å")

if __name__ == "__main__":
    main()


ModuleNotFoundError: No module named 'biopython'